# Table of Content

0. [Setup](#0.-Setup)
1. [Feature Extraction](#1.-Feature-Extractions)
2. [Convert Features into MIDI file](#2.-Convert-Features-into-MIDI-file)
3. [Convert MIDI file into Sheet Music](#3.-Convert-MIDI-file-into-Sheet-Music)

# 0. Setup

This file requires the following to be installed in your machine:
- [MuseScore](https://musescore.org/en)
- [Lilypond](https://lilypond.org/download.html)

Please also install the following libraries that are required:
- Pandas
- NumPy
- Librosa
- Matplotlib
- midiutil
- music21

In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install librosa
# !pip install matplotlib
# !pip install midiutil
# !pip install music21

In [ ]:
import pandas as pd
import numpy as np

# 1. Feature Extractions

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [ ]:
# https://www.linkedin.com/pulse/exploring-librosa-comprehensive-guide-audio-feature-extraction-m/

# Load the WAV file

audio_file = "data/en001a.wav"

y, sr = librosa.load(audio_file, sr=None)  # sr=None to preserve the original sampling rate

# Extract MFCCs

mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # You can adjust the number of MFCC coefficients (n_mfcc)

# Visualize the MFCCs

plt.figure(figsize=(10, 4))

librosa.display.specshow(librosa.power_to_db(mfccs, ref=np.max), y_axis='mel', x_axis='time')

plt.colorbar(format='%+2.0f dB')

plt.title('MFCC')

plt.show()

In [ ]:
# https://www.linkedin.com/pulse/exploring-librosa-comprehensive-guide-audio-feature-extraction-m/


# Load the WAV file
audio_file = "data/en001a.wav"
y, sr = librosa.load(audio_file, sr=None)  # sr=None to preserve the original sampling rate

# Extract MFCCs
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # You can adjust the number of MFCC coefficients (n_mfcc)

# Save MFCCs as a CSV file
mfccs_df = pd.DataFrame(mfccs)
# mfccs_df.to_csv("mfccs.csv", index=False)

# Visualize the MFCCs
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(librosa.power_to_db(mfccs, ref=np.max), y_axis='mel', x_axis='time')
# plt.colorbar(format='%+2.0f dB')
# plt.title('MFCC')
# plt.show()

mfccs_df

In [ ]:
pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
# beat_times = librosa.frames_to_time(frames,sr=sr)
librosa.display.waveshow(y, sr=sr, alpha=0.25)

In [ ]:
S = np.abs(librosa.stft(y))
chroma = librosa.feature.chroma_stft(S=S, sr=sr)


fig, ax = plt.subplots(nrows=2, sharex=True)
img = librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),
                               y_axis='log', x_axis='time', ax=ax[0])
fig.colorbar(img, ax=[ax[0]])
ax[0].label_outer()
img = librosa.display.specshow(chroma, y_axis='chroma', x_axis='time', ax=ax[1])
fig.colorbar(img, ax=[ax[1]])

# 2. Convert Features into MIDI file

In [ ]:
from midiutil import MIDIFile

In [ ]:
# degrees  = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number
# track    = 0
# channel  = 0
# time     = 0   # In beats
# duration = 1   # In beats
# tempo    = 60  # In BPM
# volume   = 100 # 0-127, as per the MIDI standard

# MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
#                      # automatically created)
# MyMIDI.addTempo(track,time, tempo)

# for pitch in degrees:
#     MyMIDI.addNote(track, channel, pitch, time, duration, volume)
#     time = time + 1

# with open("midi-output/en001a.mid", "wb") as output_file:
#     MyMIDI.writeFile(output_file)

# 3. Convert MIDI file into Sheet Music

In [ ]:
from music21 import converter, stream

# Step 1: Load the MIDI file
midi_file_path = "midi-output/major-scale.mid"
midi_stream = converter.parse(midi_file_path)

# Step 2: Convert MIDI file into a music21 Score object
score = stream.Score()
for part in midi_stream.parts:
    score.insert(0, part)

# Step 3: Show or save the Score object as sheet music
# You can either show the score
# score.show()

# Or save it as a PDF or other formats
ly_file_path = "lilypond-export/sheet_music.ly"
score.write('lilypond', ly_file_path)

# Step 4: Convert LilyPond file to PDF or SVG using LilyPond software
# You can do this manually using LilyPond software or command line
# Or you can use a library or tool to automate the conversion process
# For example, you can use the `subprocess` module to run LilyPond from Python

# Example using subprocess (requires LilyPond installed on your system):
import subprocess
output_file_name = "sheet-music-export/sheet_music"
subprocess.run(['lilypond', '--pdf', '-o', output_file_name, ly_file_path])  # Convert to PDF

# 4. Testing on en001a.wav

In [ ]:
# get pitches

import librosa
import numpy as np

# Load the audio file
y, sr = librosa.load('data/en001a.wav')

# Extract pitch and magnitude using Harmonic-Percussive Source Separation (HPSS)
pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

# Get the mean pitch over time
mean_pitches = np.mean(pitches, axis=0)

# Plot the pitch contour
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(librosa.times_like(mean_pitches), mean_pitches, label='Pitch (Hz)')
plt.xlabel('Time (s)')
plt.ylabel('Pitch (Hz)')
plt.title('Pitch Contour')
plt.legend()
plt.show()

In [ ]:
# convert each pitch into a midi note

import numpy as np

# Function to convert pitch values to MIDI notes
def pitch_to_midi(pitch):
    if pitch == 0:
        return 0
    result = 69 + 12 * np.log2(pitch / 440)
    if result < 0:
        return 0
    return result

# Example usage
# pitch_in_hz = 440  # Example pitch value in Hz
# midi_note = pitch_to_midi(pitch_in_hz)
# print("MIDI note:", midi_note)

midi_pitches = []

for pitch in mean_pitches:
    midi_pitches.append(int(pitch_to_midi(pitch)))
    
plt.figure(figsize=(10, 4))
plt.plot(librosa.times_like(np.asarray(midi_pitches)), midi_pitches, label='Pitch (Midi Notes)')
plt.xlabel('Time (s)')
plt.ylabel('Pitch (Midi Notes)')
plt.title('Pitch Contour')
plt.legend()
plt.show()

In [ ]:
from midiutil import MIDIFile

In [ ]:
degrees  = midi_pitches
track    = 0
channel  = 0
time     = 0   # In beats
duration = 1   # In beats
tempo    = 100  # In BPM
volume   = 100 # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
                     # automatically created)
MyMIDI.addTempo(track,time, tempo)

for pitch in degrees:
    MyMIDI.addNote(track, channel, pitch, time, duration, volume)
    time = time + 1

with open("midi-output/en001a.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

In [ ]:
from music21 import converter, stream

# Step 1: Load the MIDI file
midi_file_path = "midi-output/en001a.mid"
midi_stream = converter.parse(midi_file_path)

# Step 2: Convert MIDI file into a music21 Score object
score = stream.Score()
for part in midi_stream.parts:
    score.insert(0, part)

# Step 3: Show or save the Score object as sheet music
# You can either show the score
# score.show()

# Or save it as a PDF or other formats
ly_file_path = "lilypond-export/en001a_sheet_music.ly"
score.write('lilypond', ly_file_path)

# Step 4: Convert LilyPond file to PDF or SVG using LilyPond software
# You can do this manually using LilyPond software or command line
# Or you can use a library or tool to automate the conversion process
# For example, you can use the `subprocess` module to run LilyPond from Python

# Example using subprocess (requires LilyPond installed on your system):
import subprocess
output_file_name = "sheet-music-export/en001a_sheet"
subprocess.run(['lilypond', '--pdf', '-o', output_file_name, ly_file_path])  # Convert to PDF